# **Procesamiento del lenguaje natural**

------


El procesamiento del lenguaje natural (PLN) es la manipulación automática del lenguaje natural (como textos, discursos, etc.) por medio de algoritmos construidos para tal fin.

## Palabras
--------------

La mayoría de los modelos de procesamiento del lenguaje natural se basan en el estudio de las palabras. Esto se debe principalmente a que

- Las **palabras** se pueden considerar como la **unidad básica** del lengugaje.

- Existen **relaciones semánticas** entre las palabras que permiten pensar en relaciones de **similitud** y **distancia** entre las mismas.

Por supuesto, no son las únicas alternativas. El lenguaje se podría pensar también analizando conjuntos de palabras, como frases o documentos enteros, o fragmentos de palabras.

# **Extracción de Tokens**

-------
Lo primero que vamos a hacer es construir una lista de términos o **tokens** a partir de un dado texto.

In [ ]:
text = (
    'Muchos años después, frente al pelotón de fusilamiento, '
    'el coronel Aureliano Buendía había de recordar aquella tarde remota '
    'en que su padre lo llevó a conocer el hielo.'
)

## Transformamos el string en una lista de palabras
tokens = text.split()
tokens[:12]

['Muchos',
 'años',
 'después,',
 'frente',
 'al',
 'pelotón',
 'de',
 'fusilamiento,',
 'el',
 'coronel',
 'Aureliano',
 'Buendía']

Vemos que esta implementación naïve funciona relativamente bien, pero tiene algunos problemas. Por ejemplo, no reconoce los signos de puntuación.

In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')

doc = nlp(text)
tokens = [token.text for token in doc]
tokens[:14]

OSError: ignored

## Categoría gramatical (Part of Speech)

-----------

Una parte importante del procesamiento de texto es identificar a qué parte de la oración pertenece cada palabra. Por ejemplo, identificar verbos, sustantivos, artículos, adverbios, etc.

In [ ]:
[(token.text, token.pos_) for token in nlp('a la mañana yo camino por el camino')]

[('a', 'ADP'),
 ('la', 'DET'),
 ('mañana', 'NOUN'),
 ('yo', 'PRON'),
 ('camino', 'NOUN'),
 ('por', 'ADP'),
 ('el', 'DET'),
 ('camino', 'NOUN')]

In [ ]:
pos_tags = [(token.text, token.pos_) for token in doc]
pos_tags[:14]

[('Muchos', 'DET'),
 ('años', 'NOUN'),
 ('después', 'ADV'),
 (',', 'PUNCT'),
 ('frente', 'NOUN'),
 ('al', 'ADP'),
 ('pelotón', 'NOUN'),
 ('de', 'ADP'),
 ('fusilamiento', 'NOUN'),
 (',', 'PUNCT'),
 ('el', 'DET'),
 ('coronel', 'NOUN'),
 ('Aureliano', 'PROPN'),
 ('Buendía', 'PROPN')]

## Lematización

----------------

En algunos casos, muchas palabras similares corresponden a un mismo concepto. Por ejemplo, en el español los adjetivos declinan según género y número ("mucha", "mucho", "muchas", "muchos")

In [ ]:
lemmas = [(token.text, token.lemma_) for token in doc]
lemmas[:14]

[('Muchos', 'mucho'),
 ('años', 'año'),
 ('después', 'después'),
 (',', ','),
 ('frente', 'frente'),
 ('al', 'al'),
 ('pelotón', 'pelotón'),
 ('de', 'de'),
 ('fusilamiento', 'fusilamiento'),
 (',', ','),
 ('el', 'el'),
 ('coronel', 'coronel'),
 ('Aureliano', 'Aureliano'),
 ('Buendía', 'Buendía')]

## Extracción de términos relevantes

------------

Cuando trabajamos con palabras, nos interesan principalmente aquellas que representen conceptos relevantes. Por eso, es necesario excluir de nuestro análisis palabras muy frecuentes, como por ejemplo los artículos. También es conveniente descartar los signos de puntuación.

In [ ]:
def is_relevant_token(token):
    if token.is_punct or token.is_stop:
        return False
    return True

def extract_relevant_tokens(text):
    return [token for token in nlp(text) if is_relevant_token(token)]

extract_relevant_tokens(text)

[años,
 frente,
 pelotón,
 fusilamiento,
 coronel,
 Aureliano,
 Buendía,
 recordar,
 remota,
 padre,
 llevó,
 a,
 hielo]

Si ahora lematizamos, obtenemos una representación simplificada de nuestro texto original

In [ ]:
def lemmatize(text):
    return [token.lemma_ for token in nlp(text) if is_relevant_token(token)]

lemmatize(text)

['año',
 'frente',
 'pelotón',
 'fusilamiento',
 'coronel',
 'Aureliano',
 'Buendía',
 'recordar',
 'remoto',
 'padre',
 'llevar',
 'a',
 'hielo']

## Reconocimiento de entidades nombradas

-------------------

Una entidad nombrada representa un objeto concreto del mundo real, como personas, organizaciones, lugares. La librería `spaCy` nos permite extraer estas entidades del texto

In [ ]:
entities = [(ent.text, ent.label_) for ent in doc.ents]
entities

[('Aureliano Buendía', 'PER')]

# **Representaciones de palabras (embeddings)**

-------

En Machine Learning, llamamos **embedding** a la representación de un objeto como un vector. Dado que venimos trabajando con palabras, buscaremos la manera de representar a las mismas como vectores.

## One-hot encoding

Es el embedding más sencillo de construir. Partimos de un cuerpo de texto, o *corpus*, y contamos la cantidad $N$ de palabras distintas. Esas palabras distintas constituyen nuestro alfabeto, o diccionario. Luego, a cada palabra distinta le asignamos un número $i$ entre $0$ y $N-1$, y le asociamos un vector con un $1$ en la posición $i$ y ceros en las demás posiciones.



In [ ]:
def get_one_hot_embedding(text):
    tokens = text.split(' ')
    unique_words = set(text.split(' '))
    N = len(unique_words)
    word_to_int = {word: i for i, word in enumerate(unique_words)}
    word_vectors = []
    for word in tokens:
        word_vec = [0] * N
        i = word_to_int[word]
        word_vec[i] = 1
        word_vectors.append(word_vec)
    return word_to_int, word_vectors

In [ ]:
text = "Muchos años después, frente al pelotón"
word_to_int, word_vectors = get_one_hot_embedding(text)
print(word_to_int)
word_vectors

{'pelotón': 0, 'al': 1, 'Muchos': 2, 'años': 3, 'frente': 4, 'después,': 5}


[[0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0]]

Como contraparte de su sencillez, este método tiene muchas desventajas:

- Los vectores generados son ralos (tienen muchos ceros)

- Si el corpus es extenso, los vectores son muy grandes

- No refleja similitud ni cercanía de palabras

- No identifican polisemia (misma palabra con más de un significado)

## Las palabras y su contexto

El principal problema del one-hot encoding es que trata a las palabras en forma aislada, sin tener en cuenta su contexto.


Ejemplo: (sacado de [acá](https://lena-voita.github.io/nlp_course/word_embeddings.html))

¿Qué significa <span style="color:green"><strong>**tezgüino**</strong></span>?

Ni idea.

Agreguemos algo de contexto:

- Hay una botella de  <span style="color:green"><strong>**tezgüino**</strong></span> sobre la mesa.

- A todo el mundo le gusta el  <span style="color:green"><strong>**tezgüino**</strong></span>.

- El  <span style="color:green"><strong>**tezgüino**</strong></span> te emborracha.

- El <span style="color:green"><strong>**tezgüino**</strong></span> se hace con maiz.

Ahora podemos tener una idea de lo que significa la palabra (probablemente algún tipo de bebida alcohólica).

## Matriz de coocurrencia

La matriz de coocurrencia es una matriz que permite incorporar la idea de contexto a nuestro análisis. Es una simétrica donde cada fila corresponde a una palabra, y donde el elemento $i,j$ indica la cantidad de veces en las que la palabra $i$ y la palabra $j$ ocurren cerca en un texto. Típicamente, "cerca" significa una ventana de tamaño fijo centrada en la palabra.

In [ ]:
import pandas as pd
import numpy as np

def coocurrence_matrix(text, w):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct]
    token2id = {token: i for i, token in enumerate(set(tokens))}
    N = len(set(tokens))
    matrix = np.zeros((N,N), dtype='int')
    for idx, token in enumerate(tokens):
        left_idx = max(0, idx-w)
        right_idx = min(len(tokens)-1, idx+w)
        window = tokens[left_idx:right_idx]
        assert 0 < len(window) <= 2*w+1
        for neighbor in window:
            i, j = token2id[token], token2id[neighbor]
            matrix[i,j] += 1
            if i != j:
                matrix[j,i] += 1
    df = pd.DataFrame(matrix, columns=token2id.keys(), index=token2id.keys())
    return df

In [ ]:
text = (
'beautiful is better than ugly '
'explicit is better than implicit '
'simple is better than complex '
'complex is better than complicated '
'flat is better than nested '
'sparse is better than dense '
)

df = coocurrence_matrix(text, w=2)
print('Matriz de coocurrencia')
df.iloc[:7, :7]

Matriz de coocurrencia


,simple,complex,dense,flat,than,sparse,explicit
simple,1,0,0,0,1,0,0
complex,0,4,0,0,3,0,0
dense,0,0,0,0,1,0,0
flat,0,0,0,1,1,0,0
than,1,3,1,1,6,1,1
sparse,0,0,0,0,1,1,0
explicit,0,0,0,0,1,0,1


Existen formas de normalizar esta matriz, teniendo en cuenta no sólo la coocurrencia de cada palabra, sino la frecuencia con la que es utilizada. Dos métodos usules de normalización son [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) y [PMI](https://en.wikipedia.org/wiki/Pointwise_mutual_information).

La matriz de coocurrencia no resuelve el problema de la dimensión del embedding. Al igual que el one-hot encoding, la dimensión de cada vector es igual a la cantidad de palabras distintas. Esto se puede resolver haciendo una descomposición en valores singulares truncada, quedándonos con una cantidad acotada de vectores singulares.

![img](https://miro.medium.com/max/700/1*0RbLKZe8HLg9-asCsHp_Jw.png)

## Matriz de términos-documentos

Supongamos que podemos dividir nuestro corpus en documentos. La definición de documento es laxa, y podría indicar un texto completo, o bien un párrafo o una oración. Para construir la matriz de términos-documentos colocamos cada palabra como fila y cada documento como columna. Así, el elemento $i,j$ de la matriz indica la cantidad de veces que la palabra $j$ ocurre dentro del documento $j$.

![img](https://qph.fs.quoracdn.net/main-qimg-27639a9e2f88baab88a2c575a1de2005)

## Word2Vec

Word2Vec es un algoritmo de embedding en el cual se utiliza el contexto para generar una tarea predictiva, conocida como tarea de pretexto.
   
Lo primero que hacemos es deslizar una ventana sobre el texto, de manera tal de que, para cada palabra, observemos $w$ palabras hacia atrás y $w$ palabras hacia adelante. De esta manera, nos construimos un dataset para una tarea autosupervizada, que consiste en predecir la palabra central usando como features las palabras del contexto.

In [ ]:
def slide_window(text, w):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct]
    rows = []
    for i in range(w, len(tokens)-w):
        row = []
        for j in range(-w, w+1):
            if j == 0:
                continue
            row.append(tokens[i+j])
        row.append(tokens[i])
        rows.append(row)
    cols = list(range(-w, 0)) + list(range(1, w+1)) + ['target']
    df = pd.DataFrame(rows, columns=cols)
    return df

In [ ]:
text = (
'beautiful is better than ugly '
'explicit is better than implicit '
'simple is better than complex '
'complex is better than complicated '
'flat is better than nested '
'sparse is better than dense '
)

print("Nuestro dataset construido:")
df = slide_window(text, w=2)
df.head()

Nuestro dataset construido:


,-2,-1,1,2,target
0,beautiful,is,than,ugly,better
1,is,better,ugly,explicit,than
2,better,than,explicit,is,ugly
3,than,ugly,is,better,explicit
4,ugly,explicit,better,than,is


Ahora, para cada palabra asociamos un vector de dimensión dada (por ejemplo, 300), inicialmente con valores aleatorios. Luego entrenamos el modelo, ajustando los valores de los vectores de manera tal de mejorar las predicciones. Para una explicación más detallada del modelo, ver este [link](https://jalammar.github.io/illustrated-word2vec/).

Veamos cómo funciona el modelo Word2Vec en español. Usaremos el modelo entrenado por Cristian Cardellino (ver [referencia](https://crscardellino.ar/resources/nlp/2016/02/06/spanish-billion-words-corpus-and-embeddings.html)), el cual fue entrenado utilizando más de mil millones de palabras.

In [ ]:
def import_vectors():
    from gensim.models import KeyedVectors
    path = '/home/nahuel/Downloads/vectors/sbwce.wordvectors.all.bin'
    vectors = KeyedVectors.load_word2vec_format(path, binary=True)
    return vectors

In [ ]:
vectors = import_vectors()

In [ ]:
vectors.most_similar('asno')

[('burro', 0.8029149770736694),
 ('borrico', 0.7216041088104248),
 ('pollino', 0.7184825539588928),
 ('buey', 0.6988316178321838),
 ('camello', 0.6951327919960022),
 ('mulo', 0.6928654909133911),
 ('caballo', 0.6903840899467468),
 ('cebro', 0.6893405914306641),
 ('morueco', 0.6792659163475037),
 ('jumento', 0.6788411140441895)]

## Aritmética de vectores

Los vectores generados por Word2Vec permiten establecer ciertas relaciones aritméticas entre palabras. Por ejemplo, si a la palabra "rey" le resto la palabra "hombre", el resultado es similar al de restar "mujer" a "reina". Es decir,

$$
\text{rey} - \text{hombre} \approx \text{reina} - \text{mujer},
$$

o bien,


$$
\text{rey} + \text{mujer} - \text{hombre} \approx \text{reina}.
$$

(para más detalles, ver ecuación 4 en este [artículo](https://aclanthology.org/W14-1618/)).

In [ ]:
vectors.most_similar_cosmul(positive=['rey', 'mujer'], negative=['hombre'])

[('reina', 0.9791632294654846),
 ('princesa', 0.9376253485679626),
 ('consorte', 0.9167759418487549),
 ('Isabel_de_Francia', 0.9086166620254517),
 ('Juana_de_Castilla', 0.9066565632820129),
 ('Margarita_I_de_Dinamarca', 0.8999430537223816),
 ('Constanza_de_Sicilia', 0.8989415764808655),
 ('Alfonso_V_de_León', 0.8985891342163086),
 ('Sancha_de_León', 0.8979201912879944),
 ('esposa', 0.8978815674781799)]

De forma similar, se pueden verificar otro tipo de asociaciones, como por ejemplo identificar términos no relacionados dentro de una lista

In [ ]:
vectors.doesnt_match(['blanco', 'azul', 'rojo', 'argentina'])

'argentina'

Existen muchos tipos de embeddings predictivos. Algunos de ellos han sido entrenados con texto en español y pueden encontrarse en este [repositorio](https://github.com/dccuchile/spanish-word-embeddings).

# Detección de tópicos

-----------

Una de las tareas más frecuentes en el procesamiento del lenguaje natural es la detección automática de tópicos en un conjunto de documentos. Este problema puede formularse dentro del aprendizaje supervisado si contamos con etiquetas para ciertos documentos. Sin embargo, es más habitual formularlo como un problema de aprendizaje no supervisado

Uno de los métodos más empleados para detección de tópicos se conoce como análisis de semántica latente, o [LSA](https://en.wikipedia.org/wiki/Latent_semantic_analysis), por sus siglas en inglés. Básicamente consiste en utilizar la técnica de descomposición en valores singulares sobre la matriz de términos-documentos.

Para más detalles de implementación de LSA usando `gensim`, ver este [tutorial](https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python).